In [ ]:
'''
사전 학습 모델 BERT 불러와서 가지고 있는 데이터로 파인 튜닝

파인 튜닝 
ex) 사전 학습 (초등 ~ 고등 교육) -> 파인튜닝 (대학전공)

사전 훈련된 임베딩 -> 벡터 공간에 단어들이 벡터로 표현 근처의 단어 벡터들과 유사

사전 훈련된 모델 -> 문장 가지고 학습 
'''

In [ ]:
pip install sentencepiece

In [57]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [58]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x2b4cc2b1b90>)

In [59]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [60]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [61]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')


  3%|█▋                                                              | 38976512/1472569832 [00:50<18:00, 1326458.45B/s]

In [ ]:
'''
input : 학습시킬 파일
model_prefix : 만들어질 모델 이름
vocab_size : 단어 집합의 크기
model_type : 사용할 모델 (unigram(default), bpe, char, word)
max_sentence_length: 문장의 최대 길이
pad_id, pad_piece: pad token id, 값
unk_id, unk_piece: unknown token id, 값
bos_id, bos_piece: begin of sentence token id, 값
eos_id, eos_piece: end of sequence token id, 값
user_defined_symbols: 사용자 정의 토큰
'''

In [62]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE) 
# quoting=csv.QUOTE_NONE : CSV 파일을 작성하거나 읽을 때 따옴표를 사용하지 않도록 지정하는 것을 의미합니다.
vocab_list.sample(10)

,0,1
4252,▁Ac,-4249
1208,▁Ste,-1205
547,▁cont,-544
1441,het,-1438
3389,vert,-3386
2009,▁team,-2006
808,▁bu,-805
4143,▁opera,-4140
2284,▁attack,-2281
4589,▁Hall,-4586


In [63]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [64]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]

In [65]:
sp.encode_as_pieces(lines[0]) # 문장 ->  서브 워드로 변환

['▁I',
 '▁didn',
 "'",
 't',
 '▁at',
 '▁all',
 '▁think',
 '▁of',
 '▁it',
 '▁this',
 '▁way',
 '.']

In [66]:
sp.encode_as_ids(lines[0]) # 문장 -> 정수 코드로 변환

[41, 624, 4950, 4926, 139, 170, 378, 30, 58, 73, 413, 4945]

In [67]:
sp.PieceToId('▁I')

41

In [68]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x2b4ccd50b90>)

In [69]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [70]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [71]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 199992


In [72]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [73]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [74]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,ᄏᄏ,-6


In [75]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [76]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]

In [77]:
print(sp.encode_as_pieces(lines[0]))

['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']


In [78]:
print(sp.encode_as_pieces(lines[1]))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']


In [79]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [80]:
sp.DecodeIds([54, 204, 825, 121])

'진짜 최고의 영화입니다 ᄏᄏ'

In [81]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 204, 825, 121]


In [82]:
# BERT

In [83]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Bert-base의 토크나이저

In [84]:
tokenizer.vocab_size

30522

In [85]:
tokenizer.vocab

OrderedDict([('[PAD]', 0),
             ('[unused0]', 1),
             ('[unused1]', 2),
             ('[unused2]', 3),
             ('[unused3]', 4),
             ('[unused4]', 5),
             ('[unused5]', 6),
             ('[unused6]', 7),
             ('[unused7]', 8),
             ('[unused8]', 9),
             ('[unused9]', 10),
             ('[unused10]', 11),
             ('[unused11]', 12),
             ('[unused12]', 13),
             ('[unused13]', 14),
             ('[unused14]', 15),
             ('[unused15]', 16),
             ('[unused16]', 17),
             ('[unused17]', 18),
             ('[unused18]', 19),
             ('[unused19]', 20),
             ('[unused20]', 21),
             ('[unused21]', 22),
             ('[unused22]', 23),
             ('[unused23]', 24),
             ('[unused24]', 25),
             ('[unused25]', 26),
             ('[unused26]', 27),
             ('[unused27]', 28),
             ('[unused28]', 29),
             ('[unused29]', 30),
  

In [86]:
tokenizer.vocab['do']

2079

In [87]:
tokenizer.vocab['love']

2293

In [88]:
tokenizer.vocab['loves']

7459

In [89]:
tokenizer.vocab['embeddings'] # tokenize vocab에 없음

KeyError: 'embeddings'

In [90]:
tokenizer.vocab['em']

7861

In [91]:
tokenizer.vocab['##bed']

8270

In [92]:
tokenizer.vocab['##ding']

4667

In [93]:
tokenizer.vocab['##s']

2015

In [94]:
tokenizer.tokenize('Here is the sentence I want embeddings for.')

['here',
 'is',
 'the',
 'sentence',
 'i',
 'want',
 'em',
 '##bed',
 '##ding',
 '##s',
 'for',
 '.']

In [95]:
with open('vocabulary.txt', 'w', encoding='utf8') as f:
    for token in tokenizer.vocab.keys():
        f.write(token + '\n')

In [96]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [97]:
'''
별도로 약속되어 있는 특별 토큰

[PAD] - 0 # 패딩
[UNK] - 100 # 언노운
[CLS] - 101
[SEP] - 102 # 문장구분 세퍼레이트
[MASK] - 103 # 마스크
'''

'\n별도로 약속되어 있는 특별 토큰\n\n[PAD] - 0 # 패딩\n[UNK] - 100 # 언노운\n[CLS] - 101\n[SEP] - 102 # 문장구분 세퍼레이트\n[MASK] - 103 # 마스크\n'